In [1]:
!git clone https://github.com/mukuakhilesh/COVID_testCentres_India.git

Cloning into 'COVID_testCentres_India'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 1), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('IndianCOVID_testCentres_raw.csv')
df.head()

,HotspotID,LabName,Addresses
0,3,"Regional Medical Research Centre (ICMR), Port ...","Post Bag 13, Dollygunj, Port Blair, Andaman an..."
1,4,Sri Venkateswara Institute of Medical Sciences...,Sri Venkateswara Institute of Medical Sciences...
2,5,"Sidhartha Medical College, Vijayawada","Siddhartha Medical College, Ring Road, Gunadal..."
3,6,"Government Medical College, Anantapur","Government Medical College, Anantapur, Rahamat..."
4,8,"Rangaraya Medical College, Kakinada","Rangaraya Medical College, Pithampuram Road, K..."


In [7]:
# Checking if any data is repeated
df.duplicated(subset=None, keep='first').sum()

0

In [8]:
# Check for presence of invalid data
df.isna().sum()

HotspotID    0
LabName      0
Addresses    0
dtype: int64

In [9]:
# Here we separate the name of cities from different centres
city = []
for i in range(len(df)) :
  if len(df['LabName'][i].split(','))>1 :
    city.append(df['LabName'][i].split(',')[-1])
  else:
    city.append(None)
    print(i)

56
60


In [10]:
#The above two indexes were failed to parse or its city name is not there. So we put it mannualy.
print(df['LabName'][56])
print(df['LabName'][60])

Indira Gandhi Government Medical College & Hospital
Govt Medical College & Hospital Aurangabad


In [11]:
city[56] = 'Nagpur'
city[60]='Aurangabad'
city_df = pd.DataFrame(city , columns=['City'])

In [12]:
city_df.head()

,City
0,Port Blair
1,Tirupati
2,Vijayawada
3,Anantapur
4,Kakinada


In [13]:
nw_df = pd.concat([df , city_df],axis=1)

In [14]:
nw_df.to_csv('Test_centres_cities.csv')

In [15]:
!pip install opencage

In [17]:
# Here we try to geocode the available addresses using the free API from OpenCage. 

import sys
from opencage.geocoder import OpenCageGeocode

key = '2d3f2e96de7c4a2ba3102e054959f247'
geocoder = OpenCageGeocode(key)


Latitude = []
Longitude = []


for i in range(len(df)):
  query = df['Addresses'][i]
  results = geocoder.geocode(query)
  if(len(results)>0 and results[0]['components']['country_code']=='in'):         
# To check is a valid data is returned and we cross check country code to India as some addresses get mapped to foreign geolocation
    Latitude.append(results[0]['geometry']['lat'])
    Longitude.append(results[0]['geometry']['lng'])
#     print(u'%i; %f; %f; %s; %s' % ( i ,results[0]['geometry']['lat'], 
#                             results[0]['geometry']['lng'],
#                             results[0]['components']['country_code'],
#                             results[0]['annotations']['timezone']['name']))
  else:
    Latitude.append(None)
    Longitude.append(None) # Also it is diificult to geoCode some addresses so we enter it as None in dataFrame. In future it would be neede to enter manually.

In [18]:
print(len(Latitude) , len(Longitude) , len(df))

181 181 181


In [19]:
# Appending all data in one dataframe.

lats = pd.DataFrame(Latitude , columns=['Latitude'])
lng = pd.DataFrame(Longitude , columns=['Longitude'])

nw_df = pd.concat([nw_df , lats , lng ],axis=1)

In [20]:
nw_df.head()

,HotspotID,LabName,Addresses,City,Latitude,Longitude
0,3,"Regional Medical Research Centre (ICMR), Port ...","Post Bag 13, Dollygunj, Port Blair, Andaman an...",Port Blair,11.66667,92.75000
1,4,Sri Venkateswara Institute of Medical Sciences...,Sri Venkateswara Institute of Medical Sciences...,Tirupati,17.83333,79.08333
2,5,"Sidhartha Medical College, Vijayawada","Siddhartha Medical College, Ring Road, Gunadal...",Vijayawada,15.83333,79.75000
3,6,"Government Medical College, Anantapur","Government Medical College, Anantapur, Rahamat...",Anantapur,14.67940,77.59880
4,8,"Rangaraya Medical College, Kakinada","Rangaraya Medical College, Pithampuram Road, K...",Kakinada,16.96040,82.23810


In [21]:
nw_df.isna().sum()

HotspotID     0
LabName       0
Addresses     0
City          0
Latitude     41
Longitude    41
dtype: int64

In [23]:
nw_df.to_csv('TestCentres_with_geolocation.csv',index=False)
nw_df.to_excel('IndiaTestCentres_with_geolocation.xlsx',index=False)